In [1]:
import numpy as np
import os
import numba
import cupy as cp
#l=np.load('arr.npy')
#print(l)
# l=np.load('gradient.npy')
# print(len(l[0]))
from scipy.sparse import csr_matrix, save_npz, load_npz

In [2]:
agg6=np.array(np.load("err6_in_topK.npy")) #, allow_pickle=True))
agg6=agg6.transpose()
A_conc6=np.load("A_conc6_in_topK.npy", allow_pickle=True)                  #np.load("A_conc1_in.npy")
A_con=[]
iter_no=500
for i in range(iter_no):
    A_con.append(A_conc6[i].todense())
A_conc6=np.array(A_con)
A_conc6= np.concatenate(A_conc6, axis=1)
#np.save("agg3.npy",agg3)

# del err1
# del err2
# del err3

In [ ]:
A=A_conc6[:, :1000]
# print(A.shape)
# print(A)
# print(A_conc1[1862324][0:1000])
A_conc6=A

In [3]:
agg6_2=np.array(np.load("err6_in_2_topK.npy")) #, allow_pickle=True))
agg6_2=agg6_2.transpose()
A_conc6_2=np.load("A_conc6_in_2_topK.npy", allow_pickle=True)                  #np.load("A_conc1_in.npy")
A_con=[]
iter_no=500
for i in range(iter_no):
    A_con.append(A_conc6_2[i].todense())
A_conc6_2=np.array(A_con)
A_conc6_2= np.concatenate(A_conc6_2, axis=1)

In [ ]:
agg6_3=np.array(np.load("err6_in_3_topK.npy")) #, allow_pickle=True))
agg6_3=agg6_3.transpose()
A_conc6_3=np.load("A_conc6_in_3_topK.npy", allow_pickle=True)                  #np.load("A_conc1_in.npy")
A_con=[]
iter_no=500
for i in range(iter_no):
    A_con.append(A_conc6_3[i].todense())
A_conc6_3=np.array(A_con)
A_conc6_3= np.concatenate(A_conc6_3, axis=1)

In [ ]:
agg6_4=np.array(np.load("err6_in_4_topK.npy")) #, allow_pickle=True))
agg6_4=agg6_4.transpose()
A_conc6_4=np.load("A_conc6_in_4_topK.npy", allow_pickle=True)                  #np.load("A_conc1_in.npy")
A_con=[]
iter_no=500
for i in range(iter_no):
    A_con.append(A_conc6_4[i].todense())
A_conc6_4=np.array(A_con)
A_conc6_4= np.concatenate(A_conc6_4, axis=1)

In [4]:
agg6 = np.concatenate((agg6, agg6_2), axis=1) #, agg6_3, agg6_4, agg6_3 #, agg6_3, agg6_4
print(agg6.shape)
A_conc6 = np.concatenate((A_conc6, A_conc6_2), axis=1) #, A_conc6_3, A_conc6_4, A_conc6_3 #, A_conc6_3, A_conc6_4
print(A_conc6.shape)

(1862327, 1000)
(1862327, 5000)


In [5]:
del agg6_2
# del agg6_3
# del agg6_4
del A_conc6_2
# del A_conc6_3
# del A_conc6_4
import torch
torch.cuda.empty_cache()  

In [6]:
print( np. __version__)

1.21.0


In [7]:
import torch
if torch.cuda.is_available():
  dev = "cuda:3"
else:
  dev = "cpu"
device = torch.device(dev)

In [8]:
from numba import cuda
from numba import jit
print(cuda.gpus)
numba.cuda.select_device( 3 )

<Managed Device 0>, <Managed Device 1>, <Managed Device 2>, <Managed Device 3>


<weakproxy at 0x7fae79a5d720 to Device at 0x7fae79a577c0>

In [9]:
from numba import cuda
cuda.detect()
from numba import jit
#@cuda.jit
@jit #(target='cuda:1')
def matrix_comp(A,A_conc,d,iter_no):
    #for t in range(int(d/6)):
    for t in range(A_conc.shape[0]):
#     if (t%500000)==0:
#         print(t)
    #for j in range(user_no):
        A[t]=np.split(A_conc[t],iter_no)
        #A[t]=np.ma.row_stack (A[t])
    return A

Found 4 CUDA devices
id 0     b'NVIDIA RTX A4000'                              [SUPPORTED]
                      Compute Capability: 8.6
                           PCI Device ID: 0
                              PCI Bus ID: 1
                                    UUID: GPU-bdffda51-dada-8c75-fed9-ef75eda5d187
                                Watchdog: Enabled
             FP32/FP64 Performance Ratio: 32
id 1     b'NVIDIA RTX A4000'                              [SUPPORTED]
                      Compute Capability: 8.6
                           PCI Device ID: 0
                              PCI Bus ID: 33
                                    UUID: GPU-b38425d5-b022-2e33-6867-9135f00e0a68
                                Watchdog: Enabled
             FP32/FP64 Performance Ratio: 32
id 2     b'NVIDIA RTX A4000'                              [SUPPORTED]
                      Compute Capability: 8.6
                           PCI Device ID: 0
                              PCI Bus ID: 75
         

<ipython-input-9-fdb9d8741d67>:6: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def matrix_comp(A,A_conc,d,iter_no):


In [ ]:
import time
d=11173962
iter_no=1000
user_no=5
A6=np.empty((int(d/6),iter_no,user_no))
start=time.time()
A6=matrix_comp(A6,A_conc6,d,iter_no)
end=time.time()
print(end-start)

<ipython-input-9-fdb9d8741d67>:5: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "matrix_comp" failed type inference due to: No implementation of function Function(<built-in function setitem>) found for signature:
 
 >>> setitem(array(float64, 3d, C), int64, list(array(uint16, 1d, C))<iv=None>)
 
There are 16 candidate implementations:
  - Of which 16 did not match due to:
  Overload of function 'setitem': File: <numerous>: Line N/A.
    With argument(s): '(array(float64, 3d, C), int64, list(array(uint16, 1d, C))<iv=None>)':
   No match.

During: typing of setitem at <ipython-input-9-fdb9d8741d67> (12)

File "<ipython-input-9-fdb9d8741d67>", line 12:
def matrix_comp(A,A_conc,d,iter_no):
    <source elided>
    #for j in range(user_no):
        A[t]=np.split(A_conc[t],iter_no)
        ^

  @jit #(target='cuda:1')
<ipython-input-9-fdb9d8741d67>:5: NumbaWarning: 
Compilation is falling back to object mode WITHOUT looplifting enabled bec

In [ ]:
del A_conc6
torch.cuda.empty_cache()  

In [ ]:
from scipy import linalg
import time

#print(A)
#A=np.array(A)
#p,*_ = linalg.lstsq(A[:,:,0], e)
d=11173962

user_gradient=[]
check_gradient=[]
ind1=int(d/6)
num_users=user_no
start=time.time()   
for user in range(num_users):
#for user in user_no:
    user_gradient.append([])
    #check_gradient.append(original_gradient[user])
for t in range(ind1):
    #A_big[t]=np.array(A_big[t])
    #e[t]=np.array(e[t])
        #p,*_ = linalg.lstsq(A1[t], agg1[t])
    p, *_  = linalg.lstsq(A6[t], agg6[t][0:iter_no])
    #print(p)
    for user in range(num_users):
        user_gradient[user].append(p[user])
    
#check_gradient[user]=original_gradient[user]
end=time.time()
# ind1=int(d/3)
# estimated_grad=least_square(A1,agg1,ind1)
# end=time.time()

print(end-start)

In [ ]:
o=np.load("original_grad.npy")

In [ ]:
#o=np.load("original_grad.npy")
print(o[2][d-20:d])
print(user_gradient[2][int(d/6)-20:int(d/6)])
print(user_gradient[2][0:20])
print(np.linalg.norm(o[0][int(5*d/6):d][:,0]-user_gradient[0]))

In [ ]:
np.save("gradient66.npy",user_gradient)
print("finish")